# Entrega 2, Grupo 02 - Aprendizaje Bayesiano

- Santiago Alaniz,  5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone,  4914555-0, bruno.de.simone@fing.edu.uy
- María Usuca,      4891124-3, maria.usuca@fing.edu.uy

## Objetivo

Implementar un algoritmo de predicción de palabras utilizando Naive Bayes. El modelo se entrenará con datos de conversaciones de WhatsApp y se probará en un simulador de cliente. 

El algoritmo debe ser capaz de recomendar palabras basadas en las últimas N palabras ingresadas en una frase, donde N es un hiperparámetro que se variará para evaluar el desempeño del modelo. Además, el modelo se reentrenará al finalizar cada frase para adaptarse a nueva evidencia. 

La implementación debe ser eficiente en términos de uso de CPU, utilizando las estructuras de datos más adecuadas en Python.


## Diseño

### Preprocesamiento de datos
- Decisiones sobre tratamiento de datos numéricos, faltantes, etc. antes de la aplicación de el algoritmo
- Selección/generación de atributos


**Carga de datos y Transformación**: A continuación vamos a realizar la carga de una conversación y su transformación para:
- Obtener solo las palabras en el diccionario.
- Pasar todas las palabras a minusculas.
- Armar una lista y ordenar

In [11]:
import nltk
from nltk.corpus import words
from collections import Counter

TXT_PATH = './assets/chat_3.txt'

nltk.download('words')
spanish_words = set(words.words())

with open(TXT_PATH, 'r', encoding='utf-8') as file:
    lines = file.readlines()

filtered_content = [line.split(':', 1)[-1].strip() for line in lines]
all_words = ' '.join(filtered_content).split()
all_words_lower = [word.lower() for word in all_words]


filtered_words = [word for word in all_words_lower if word in spanish_words]

word_count = Counter(filtered_words)
sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

for word, frequency in sorted_words:
    print(f"{word}: {frequency}")

[nltk_data] Downloading package words to C:\Users\Maria
[nltk_data]     Usuca\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


la: 1518
de: 1454
el: 1368
a: 1322
y: 1213
no: 1149
en: 976
me: 846
si: 718
lo: 661
yo: 615
para: 576
se: 567
es: 450
las: 436
un: 415
ya: 412
con: 379
te: 376
o: 326
hay: 288
al: 228
hoy: 222
bien: 208
mi: 174
pa: 152
ta: 134
ni: 85
re: 85
dale: 83
son: 80
toy: 79
hasta: 78
hora: 77
dos: 66
solo: 60
gas: 57
van: 56
bo: 55
sale: 53
tu: 53
sin: 52
idea: 52
antes: 49
ah: 48
era: 46
poco: 43
dice: 43
comer: 43
da: 42
oki: 39
lindo: 39
ser: 38
che: 38
split: 38
mal: 37
loco: 35
aviso: 34
agua: 34
mas: 33
oka: 32
pal: 32
vas: 32
feria: 32
anda: 31
nose: 31
tan: 30
arriba: 28
porto: 28
medio: 27
sea: 26
claro: 25
e: 23
iba: 22
soy: 22
piso: 22
pan: 21
primer: 19
saber: 19
familia: 19
super: 19
full: 18
living: 18
sabe: 18
timbre: 18
f: 17
pizza: 17
null: 17
haya: 16
final: 16
na: 16
lunes: 16
vino: 16
usar: 16
media: 15
bus: 15
mano: 15
pinta: 14
dan: 14
tal: 14
dar: 14
chico: 14
sisi: 12
data: 12
eh: 12
maria: 12
mala: 12
pah: 12
vine: 12
tire: 12
cola: 11
base: 11
justo: 11
pronto: 11
paga

### Algoritmo
Extensiones del algoritmo original necesarias para la resolución del problema: tratamiento de atributos faltantes, numéricos, etc. (si es el propio algoritmo el que lo maneja), implementaciones adicionales necesarias para manejar ensambles de clasificadores, etc.

### Evaluación
- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.

## Experimentación

### Simulador de cliente.

El experimento consiste en evaluar los distintos hiperparametros del modelo. Alterando el tamaño de la ventana de palabras, y el vocabulario buscamos encontrar un equilibrio entre las siguientes propiedades deseables:

- **Memoria**: El modelo debe ser capaz de sugerir palabras que tengan coherencia con el grupo de Whatsapp de donde se extrajeron los datos.

- **Adaptacion**: El modelo debe ser capaz de adaptarse a la nueva evidencia, y mejorar su performance, sugiriendo palabras que tengan coherencia con la sesion de chat que se esta simulando.

El siguiente script permite simular el comportamiento de un cliente que escribe frases, fue proporcionado por el cuerpo docente. Es de utilidad para evaluar el desempeño del modelo, ya que permite ingresar frases y ver las sugerencias que el modelo realiza.


In [ ]:
def recomendacion_bayesiana(frase):
  import random

  dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

  ## PSEUDOCODIGO - pag 55 - diapo##

  D = ["mi", "hijo", "se", "olvidó", "de", "la"]
  Horizonte = 4 #hiperparametro
  h_MAP = ""
  p_MAP = 0
  for h in P:
    
    prob = P[h]
    
    for d in D[-Horizonte:]:
      prob = prob * PD[h].get(d, P_nada) 
      #P_nada es un valor que debemos definir para el caso cuando la palabra no se encuentre en el listado
    
    if prob > p_MAP:
      h_MAP , p_MAP = h , prob
  print(h_MAP)

  return(random.choice(dias))


##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""

while 1:
  palabra = input(">> ")

  if palabra == "..":
    break

  elif palabra == ".":
    print("----- Comenzando frase nueva -----")
    frase = []

  elif palabra == "": # acepta última palabra sugerida
    frase.append(palabra_sugerida)

  else: # escribió una palabra
    frase.append(palabra)

  if frase:
    palabra_sugerida = recomendacion_bayesiana(frase)

    frase_propuesta = frase.copy()
    frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

    print(" ".join(frase_propuesta))

## Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?